# Исследование данных музыкального сервиса - сравнение музыкальных предпочтений пользователей двух городов
---


**Цель исследования** — проверка трёх гипотез:
1. Активность пользователей зависит от дня недели. Причём в в разных городах это проявляется по-разному.
2. В понедельник утром в одном городе преобладают одни жанры, а в другом — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.
3. В разных городах предпочитают разные жанры музыки.

**Ход исследования**

Данные о поведении пользователей хранятся взяты с kaggle. О качестве данных ничего не известно. Поэтому перед проверкой гипотез понадобится обзор данных. 

Необходимо проверить данные на ошибки и оценить их влияние на исследование. Затем, на этапе предобработки следует поискать возможность исправить самые критичные ошибки данных.
 
Таким образом, исследование пройдёт в три этапа:
 1. Обзор данных.
 2. Предобработка данных.
 3. Проверка гипотез.

---

## Обзор данных

Составим первое представление о данных.

In [2]:
import pandas as pd

Прочитаем файлы, соедим, отберём нужные столбцы и сохраним его в переменной `songs`:

In [12]:
song_df = pd.read_csv('songs.csv')
song_ex_df = pd.read_csv('song_extra_info.csv')
user_df = pd.read_csv('members.csv')
train_df = pd.read_csv('train.csv')
s_se = pd.merge(train_df, song_df, on='song_id', how='left')
s_se_t = pd.merge(s_se, song_ex_df, on='song_id', how='left')
songs = pd.merge(s_se_t, user_df, on='msno', how='left')
songs = songs.iloc[:, [0, 12, 8, 7, 14, 6, 18]] # Оставляем только нужные столбцы
del song_df, song_ex_df, user_df, train_df, s_se, s_se_t

Для анализа возьмем города с id 8 и 9

In [33]:
songs.city.value_counts()
songs.loc[(songs.city == 8) | (songs.city == 9)]

,msno,name,artist_name,genre_ids,city,song_length,registration_init_time
19,uQQHTQJ1nVEkBfbXe0f1/J9ML5eQhsQiocCV5TvvCRI=,雨天 (雨天),孫燕姿 (Yanzi Sun),465,9,241975.0,20130912
20,uQQHTQJ1nVEkBfbXe0f1/J9ML5eQhsQiocCV5TvvCRI=,EGO-HOLIC戀我癖 (EGO-HOLIC),陳星翰 (Starr Chen),458,9,189846.0,20130912
21,uQQHTQJ1nVEkBfbXe0f1/J9ML5eQhsQiocCV5TvvCRI=,愛死你,莫文蔚 (Karen Mok),465,9,247911.0,20130912
22,uQQHTQJ1nVEkBfbXe0f1/J9ML5eQhsQiocCV5TvvCRI=,麥來亂,五月天 (Mayday),465,9,257671.0,20130912
571,Ix1RlLsCGktMc8Q5oogErdJbfKL+HYj116+bYZ4qzMc=,愛久見人心,梁靜茹 (Fish Leong),458,8,296542.0,20130722
...,...,...,...,...,...,...,...
7376782,WtuOpsYQ8St9zt7yySdryqDUCxeT4/v2rLRA7+lsahQ=,Bellas Regionals,Pitch Perfect Soundtrack,921,8,160055.0,20151227
7376783,WtuOpsYQ8St9zt7yySdryqDUCxeT4/v2rLRA7+lsahQ=,Live High,Jason Mraz,465,8,254537.0,20151227
7377115,WtuOpsYQ8St9zt7yySdryqDUCxeT4/v2rLRA7+lsahQ=,When You're Good To Mama,Chicago,465,8,199854.0,20151227
7377116,WtuOpsYQ8St9zt7yySdryqDUCxeT4/v2rLRA7+lsahQ=,Escape,MISIA,139|125|109,8,297471.0,20151227


In [6]:
songs.head()

,user_id,track,artist,genre_ids,city,song_length,day
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,Good Grief,Bastille,359,1,206471.0,20120102
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,Lords of Cardboard,Various Artists,1259,13,284584.0,20110525
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,Hip Hop Is Dead(Album Version (Edited)),Nas,1259,13,225396.0,20110525
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,Disco Africa,Soundway,1019,13,255512.0,20110525
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,Sleep Without You,Brett Young,1011,1,187802.0,20120102


In [13]:
songs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   user_id      100000 non-null  object 
 1   track        99991 non-null   object 
 2   artist       99996 non-null   object 
 3   genre_ids    98498 non-null   object 
 4   city         100000 non-null  int64  
 5   song_length  99996 non-null   float64
 6   day          100000 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 6.1+ MB


Итак, в таблице у нас получилось семь столбцов.

Согласно документации к данным:
* `msno` — идентификатор пользователя;
* `name` — название трека;  
* `artist_name` — имя исполнителя;
* `genre_ids` — id жанра;
* `city` — город пользователя;
* `song_length` — длительность песни;
* `registration_init_time` — дата первого прослушивания;


Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.

#### **Выводы**

В каждой строке таблицы — данные о прослушанном треке. Часть колонок описывает саму композицию: название, исполнителя и жанр. Остальные данные рассказывают о пользователе: из какого он города, когда он слушал музыку.

Предварительно можно утверждать, что, данных достаточно для проверки гипотез. Но встречаются пропуски в данных.

Пропуски в данных наблюдаются только в колонках, описывающих саму композицию, что может свидетельствовать об ошибках при извлечении данных о композициях, либо о том, что в системе поля Track, artist и genre_ids являются необязательными и теоретически возможна ситуация, когда все три поля будут пропущены одновременно. Целесообразно проконсультироваться с разработчиками.

С точки зрения постановки задачи наиболее важным является столбец genre_ids.

Чтобы двигаться дальше, устраним проблемы в данных.

## **Предобработка данных**
Исправим стиль в заголовках столбцов, исключим пропуски.

## Стиль заголовков
Выведем на экран названия столбцов:

In [9]:
songs.columns

Index(['user_id', 'track', 'artist', 'genre_ids', 'city', 'song_length',
       'day'],
      dtype='object')

In [8]:
# Изменим названия на более понятные.
songs = songs.rename(
    columns={
        'msno' : 'user_id',
        'name' : 'track',
        'artist_name' : 'artist',
        'registration_init_time' : 'date'
    }
)

In [11]:
# проверка результатов - перечень названий столбцов
songs.columns

Index(['user_id', 'track', 'artist', 'genre_ids', 'city', 'song_length',
       'day'],
      dtype='object')

## Работа с пропущенными значениями
Сначала посчитаем, сколько в таблице пропущенных значений:

In [10]:
# подсчёт пропусков
songs.isna().sum()

user_id           0
track             9
artist            4
genre_ids      1502
city              0
song_length       4
day               0
dtype: int64

In [8]:
for i in songs.select_dtypes(include=['object']).columns:
    songs[i][songs[i].isnull()] = 'unknown'
songs = songs.fillna(value=0)

In [9]:
songs.duplicated().sum()

0